# Build a Retrieval Augmented Generation (RAG) App

# ChatMistralAI, langchain, FAISS 

In [1]:
# ! pip install langchain langchain_community langchain_chroma
# ! pip install -U langchain-community faiss-cpu
# ! pip install -qU langchain-mistralai
# ! pip install sentence_transformers --quiet
# !pip install langchainhub

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ···················································


## using ChatMistralAI mistral-large-latest model

In [3]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest")

 ································


## using HuggingFaceEmbeddings model

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-17 08:37:03.734069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 08:37:03.734126: E external/local_x

## load Document/data

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


## Split into chunks

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(len(splits))


66


## vectorstores (Faiss,chromadb,pinecone)

In [8]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)


## Retrieval and Generation

In [12]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [13]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex problem or task into smaller, more manageable parts or subtasks. This can be achieved in several ways, such as by using a language model with simple prompting, task-specific instructions, or human inputs. For instance, in writing a novel, the task can be decomposed by creating a story outline. The execution of these subtasks is then carried out by expert models, which log the results for further analysis and inference. However, task decomposition and long-term planning can be challenging due to the limited context length and the difficulty in adjusting plans when faced with unexpected errors.'

### Customizing the prompt

In [14]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)



In [15]:
rag_chain.invoke("What is Self-Reflection brief in two or three sentance?")

"Self-reflection in this context is a process where an agent, after determining that its trajectory is inefficient or contains hallucinations, analyzes its failed trajectory and generates ideal reflections to guide future changes in its plan. These reflections are then added to the agent's working memory and used as context for querying the language model, helping the agent to optimize its actions and improve its reasoning skills. Thanks for asking!"